# 각 phase간 z방향 slice를 맞추는 루틴 파이프라인

In [1]:
import os
import glob
import natsort
from tqdm import tqdm
import SimpleITK as sitk
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

%config Completer.use_jedi = False

# 정합이 잘 되지 않은 영상의 인덱스와 환자 ID 확인하기

### artery와 delay 각각 hdr_name 확인

In [ ]:
# # Dataframe of artery case
# df_a = pd.read_csv('./failure_reg_artery_list.csv', usecols=[0])

# # Dataframe of delay case
# df_d = pd.read_csv('./failure_reg_delay_list.csv', usecols=[0])

# print(df_a)
# print('')
# print(df_d)

In [ ]:
# # 2중 리스트를 flatten하게 만들기

# a_tolist = df_a.values.tolist()
# a_flat = [y for x in a_tolist for y in x]

# d_tolist = df_d.values.tolist()
# d_flat = [y for x in d_tolist for y in x]

# len(a_flat), len(d_flat)

In [ ]:
# ad_flat = a_flat + d_flat
# len(ad_flat)

In [ ]:
# # artery와 delay 영상의 모든 hdr_name 중복 없이 종합 -> portal 영상의 hdr_name

# portal_lst = []
# for name in ad_flat:
#     if name not in portal_lst:
#         portal_lst.append(name)

# len(portal_lst)

In [ ]:
# artery_lst = a_flat
# delay_lst = d_flat

# len(artery_lst), len(portal_lst), len(delay_lst)

# liver slice만 남기기

In [2]:
ct_basepath = './'

# phase별로 ct 경로
ct_arterys = natsort.natsorted(glob.glob(os.path.join(ct_basepath, 'ct_artery/*_resize.hdr')))
ct_portals = natsort.natsorted(glob.glob(os.path.join(ct_basepath, 'ct_portal/*_resize.hdr')))
ct_delays = natsort.natsorted(glob.glob(os.path.join(ct_basepath, 'ct_delay/*_resize.hdr')))

# phase별로 mask 경로
mask_arterys = natsort.natsorted(glob.glob(os.path.join(ct_basepath, 'mask_artery/*_resize*.hdr')))
mask_portals = natsort.natsorted(glob.glob(os.path.join(ct_basepath, 'mask_portal/*_resize*.hdr')))
mask_delays = natsort.natsorted(glob.glob(os.path.join(ct_basepath, 'mask_delay/*_resize*.hdr')))

len(ct_arterys), len(ct_portals), len(ct_delays), len(mask_arterys), len(mask_portals), len(mask_delays)

(809, 808, 749, 809, 808, 749)

In [ ]:
# ct_a = []
# for path in ct_artery:
#     name = os.path.basename(path).split('_')[0]+'_'+os.path.basename(path).split('_')[1]
#     if name in artery_lst:
#             ct_a.append(path)
            
# ct_p = []
# for path in ct_portal:
#     name = os.path.basename(path).split('_')[0]+'_'+os.path.basename(path).split('_')[1]
#     if name in portal_lst:
#             ct_p.append(path)

# ct_d = []
# for path in ct_delay:
#     name = os.path.basename(path).split('_')[0]+'_'+os.path.basename(path).split('_')[1]
#     if name in delay_lst:
#             ct_d.append(path)
    
# len(ct_a), len(ct_p), len(ct_d)

In [ ]:
# mask_a = []
# for path in MASK_artery:
#     name = os.path.basename(path).split('_')[0]+'_'+os.path.basename(path).split('_')[1]
#     if name in artery_lst:
#             mask_a.append(path)
            
# mask_p = []
# for path in MASK_portal:
#     name = os.path.basename(path).split('_')[0]+'_'+os.path.basename(path).split('_')[1]
#     if name in portal_lst:
#             mask_p.append(path)

# mask_d = []
# for path in MASK_delay:
#     name = os.path.basename(path).split('_')[0]+'_'+os.path.basename(path).split('_')[1]
#     if name in delay_lst:
#             mask_d.append(path)
    
# len(mask_a), len(mask_p), len(mask_d)

#### phase 간 (artery와 portal / delay와 portal) slice 개수를 crop한 CT 영상 저장하기

##### SimpleITK.WriteImage로 저장하기 전에 Set 해줘야하는 변수들
- SetSpacing
- SetOrigin
- SetDirection

In [4]:
for i, mask_path in tqdm(enumerate(mask_arterys), total=len(mask_arterys)):
    fname = os.path.basename(ct_arterys[i]).replace('resize', 'onlyliver')
    img_artery = sitk.ReadImage(ct_arterys[i])
    img_artery_arr = sitk.GetArrayFromImage(img_artery)
    mask_artery = sitk.ReadImage(mask_path)
    mask_artery_arr = sitk.GetArrayFromImage(mask_artery)
    
    ct_artery_slices = []
    for i in range(len(mask_artery_arr)):
        if len(mask_artery_arr[i][np.where(mask_artery_arr[i] > 0)]) != 0:
            ct_artery_slices.append(img_artery_arr[i])
    
    ct_artery = np.array(ct_artery_slices)
    crop_liver_img = sitk.GetImageFromArray(ct_artery)
    crop_liver_img.SetSpacing(img_artery.GetSpacing())
    crop_liver_img.SetOrigin(img_artery.GetOrigin())
    crop_liver_img.SetDirection(img_artery.GetDirection())
    sitk.WriteImage(crop_liver_img, f'./liver_crop/artery/{fname}.hdr')

100%|██████████| 809/809 [10:37<00:00,  1.27it/s]


In [5]:
for i, mask_path in tqdm(enumerate(mask_portals), total=len(mask_portals)):
    fname = os.path.basename(ct_portals[i]).replace('resize', 'onlyliver')
    img_portal = sitk.ReadImage(ct_portals[i])
    img_portal_arr = sitk.GetArrayFromImage(img_portal)
    mask_portal = sitk.ReadImage(mask_path)
    mask_portal_arr = sitk.GetArrayFromImage(mask_portal)
    
    ct_portal_slices = []
    for i in range(len(mask_portal_arr)):
        if len(mask_portal_arr[i][np.where(mask_portal_arr[i] > 0)]) != 0:
            ct_portal_slices.append(img_portal_arr[i])
    
    ct_portal = np.array(ct_portal_slices)
    crop_liver_img = sitk.GetImageFromArray(ct_portal)
    crop_liver_img.SetSpacing(img_portal.GetSpacing())
    crop_liver_img.SetOrigin(img_portal.GetOrigin())
    crop_liver_img.SetDirection(img_portal.GetDirection())
    sitk.WriteImage(crop_liver_img, f'./liver_crop/portal/{fname}.hdr')

100%|██████████| 808/808 [14:49<00:00,  1.10s/it]


In [6]:
for i, mask_path in tqdm(enumerate(mask_delays), total=len(mask_delays)):
    fname = os.path.basename(ct_delays[i]).replace('resize', 'onlyliver')
    img_delay = sitk.ReadImage(ct_delays[i])
    img_delay_arr = sitk.GetArrayFromImage(img_delay)
    mask_delay = sitk.ReadImage(mask_path)
    mask_delay_arr = sitk.GetArrayFromImage(mask_delay)
    
    ct_delay_slices = []
    for i in range(len(mask_delay_arr)):
        if len(mask_delay_arr[i][np.where(mask_delay_arr[i] > 0)]) != 0:
            ct_delay_slices.append(img_delay_arr[i])
    
    ct_delay = np.array(ct_delay_slices)
    crop_liver_img = sitk.GetImageFromArray(ct_delay)
    crop_liver_img.SetSpacing(img_delay.GetSpacing())
    crop_liver_img.SetOrigin(img_delay.GetOrigin())
    crop_liver_img.SetDirection(img_delay.GetDirection())
    sitk.WriteImage(crop_liver_img, f'./liver_crop/delay/{fname}.hdr')

100%|██████████| 749/749 [10:44<00:00,  1.16it/s]


#### phase 간 (artery와 portal / delay와 portal) slice 개수를 crop한 마스크

In [7]:
for i, mask_path in tqdm(enumerate(mask_arterys), total=len(mask_arterys)):
    fname = os.path.basename(mask_path).replace('resize_Lg', 'onlyliver')
    mask_img = sitk.ReadImage(mask_path)
    mask_arr = sitk.GetArrayFromImage(mask_img)
    
    mask_arr_slices = []
    for i in range(len(mask_arr)):
        if len(mask_arr[i][np.where(mask_arr[i] > 0)]) != 0:
            mask_arr_slices.append(mask_arr[i])
    
    mask_artery = np.array(mask_arr_slices)
    only_liver_img = sitk.GetImageFromArray(mask_artery)
    only_liver_img.SetSpacing(mask_img.GetSpacing())
    only_liver_img.SetOrigin(mask_img.GetOrigin())
    only_liver_img.SetDirection(mask_img.GetDirection())
    sitk.WriteImage(only_liver_img, f'./mask_crop/artery/{fname}.hdr')

100%|██████████| 809/809 [03:14<00:00,  4.17it/s]


In [9]:
for i, mask_path in tqdm(enumerate(mask_portals), total=len(mask_portals)):
    fname = os.path.basename(mask_path).replace('resize_Lg', 'onlyliver')
    mask_img = sitk.ReadImage(mask_path)
    mask_arr = sitk.GetArrayFromImage(mask_img)
    
    mask_arr_slices = []
    for i in range(len(mask_arr)):
        if len(mask_arr[i][np.where(mask_arr[i] > 0)]) != 0:
            mask_arr_slices.append(mask_arr[i])
    
    mask_portal = np.array(mask_arr_slices)
    only_liver_img = sitk.GetImageFromArray(mask_portal)
    only_liver_img.SetSpacing(mask_img.GetSpacing())
    only_liver_img.SetOrigin(mask_img.GetOrigin())
    only_liver_img.SetDirection(mask_img.GetDirection())
    sitk.WriteImage(only_liver_img, f'./mask_crop/portal/{fname}.hdr')

100%|██████████| 808/808 [03:23<00:00,  3.96it/s]


In [10]:
for i, mask_path in tqdm(enumerate(mask_delays), total=len(mask_delays)):
    fname = os.path.basename(mask_path).replace('resize_Lg', 'onlyliver')
    mask_img = sitk.ReadImage(mask_path)
    mask_arr = sitk.GetArrayFromImage(mask_img)
    
    mask_arr_slices = []
    for i in range(len(mask_arr)):
        if len(mask_arr[i][np.where(mask_arr[i] > 0)]) != 0:
            mask_arr_slices.append(mask_arr[i])
    
    mask_delay = np.array(mask_arr_slices)
    only_liver_img = sitk.GetImageFromArray(mask_delay)
    only_liver_img.SetSpacing(mask_img.GetSpacing())
    only_liver_img.SetOrigin(mask_img.GetOrigin())
    only_liver_img.SetDirection(mask_img.GetDirection())
    sitk.WriteImage(only_liver_img, f'./mask_crop/delay/{fname}.hdr')

100%|██████████| 749/749 [02:49<00:00,  4.42it/s]


# Histomgram

## slice 개수 차이 맞추기 이전

In [ ]:
# before_artery = []
# for idx, path in tqdm(enumerate(ct_a), total=len(ct_a)):
#     a_img = sitk.ReadImage(path)
#     a_ary = sitk.GetArrayFromImage(a_img)
#     a_depth = a_ary.shape[0]
    
#     fname = os.path.basename(path)
#     p_path = f'/dockerspace/data2/sungchul/CDSS_Liver/data/CDSS/CT_only_portal/{fname}'
    
#     p_img = sitk.ReadImage(p_path)
#     p_ary = sitk.GetArrayFromImage(p_img)
#     p_depth = p_ary.shape[0]
    
#     dif = p_depth - a_depth
#     before_artery.append(dif)
    
# before_delay = []
# for idx, path in tqdm(enumerate(ct_d), total=len(ct_d)):
#     d_img = sitk.ReadImage(path)
#     d_ary = sitk.GetArrayFromImage(d_img)
#     d_depth = d_ary.shape[0]
    
#     fname = os.path.basename(path)
#     p_path = f'/dockerspace/data2/sungchul/CDSS_Liver/data/CDSS/CT_only_portal/{fname}'
    
#     p_img = sitk.ReadImage(p_path)
#     p_ary = sitk.GetArrayFromImage(p_img)
#     p_depth = p_ary.shape[0]

#     dif = p_depth - d_depth
#     before_delay.append(dif)

In [ ]:
# len(before_artery), len(before_delay)

In [ ]:
# plt.hist(before_artery, bins=14, range=[0,70], alpha=.7, label='artery')
# plt.legend()
# plt.hist(before_delay, bins=14, range=[0,70], alpha=.7, label='delay')
# plt.legend()
# plt.xlabel('number of z slices(portal) - number of z slices(phase)')
# plt.show()

# print('Max difference b/w artery & portal:', max(before_artery))
# print('Max difference b/w artery & delay:', max(before_delay))

## slice 개수 차이 맞추기 나서 이후

In [ ]:
# CT_a = glob.glob('./liver_crop/artery/*.hdr')
# CT_p = glob.glob('./liver_crop/portal/*.hdr')
# CT_d = glob.glob('./liver_crop/delay/*.hdr')

# len(CT_a), len(CT_p), len(CT_d)

In [ ]:
# after_artery = []
# for idx, path in tqdm(enumerate(CT_a), total=len(CT_a)):
#     a_img = sitk.ReadImage(path)
#     a_ary = sitk.GetArrayFromImage(a_img)
#     a_depth = a_ary.shape[0]
    
#     fname = os.path.basename(path)
#     p_path = f'./liver_crop/portal/{fname}'
    
#     p_img = sitk.ReadImage(p_path)
#     p_ary = sitk.GetArrayFromImage(p_img)
#     p_depth = p_ary.shape[0]
    
#     dif = p_depth - a_depth
#     after_artery.append(dif)
    
# after_delay = []
# for idx, path in tqdm(enumerate(CT_d), total=len(CT_d)):
#     d_img = sitk.ReadImage(path)
#     d_ary = sitk.GetArrayFromImage(d_img)
#     d_depth = d_ary.shape[0]
    
#     fname = os.path.basename(path)
#     p_path = f'./liver_crop/portal/{fname}'
    
#     p_img = sitk.ReadImage(p_path)
#     p_ary = sitk.GetArrayFromImage(p_img)
#     p_depth = p_ary.shape[0]

#     dif = p_depth - d_depth
#     after_delay.append(dif)

In [ ]:
# len(before_artery), len(before_delay)

In [ ]:
# plt.hist(after_artery, bins=14, range=[0,70], alpha=.7, label='artery')
# plt.legend()
# plt.hist(after_delay, bins=14, range=[0,70], alpha=.7, label='delay')
# plt.legend()
# plt.xlabel('number of z slices(portal) - number of z slices(phase)')
# plt.show()

# print('Max difference b/w artery & portal:', max(after_artery))
# print('Max difference b/w artery & delay:', max(after_delay))